In [ ]:
#Learning about noisy channels 

#For which we will have a look at the SendState function with some errors

#We will introduce a NoisyChannel function with 12.5% chance that qubit has an error  

In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer

from random import randrange

#Defining Noisy Channel

def NoisyChannel(qc1,qc2,qc1_name):
    """
    This function takes one circuit qc1 as input and then 
    introduce Pauli errors and initializes another circuit qc2
    with introduce noise
    """

    qs = qc1.qasm().split(sep = ';')[4:-1]

    for index, instruction in enumerate(qs):
        qs[index] = instruction.lstrip()

    
    for instruction in qs:
        if instruction[0] == 'x':
            if instruction[5] == '[':
                old_qr = int(instruction[6:-1])
            else:
                old_qr = int(instruction[5:-1])

            qc2.x(qreg[old_qr])

        elif instruction[0] == 'h':
            if instruction[5] == '[':
                old_qr = int(instruction[6:-1])
            else:
                old_qr = int(instruction[5:-1])
            
            qc2.h(qreg[old_qr])
        elif instruction[0] == 'm':
            pass
        else:
            raise Exception("Unable to parse instruction")
        


        #Introduction noise
        for instruction in qs:
            if randrange(7)<1:
                if instruction[5] == '[':
                    old_qr = int(instruction[6:-1])
                else:
                    old_qr = int(instruction[5:-1])
                qc2.x(qreg[old_qr]) #Applying bit flip error
            if randrange(7)<1:
                if instruction[5] == '[':
                    old_qr = int(instruction[6:-1])
                else:
                    old_qr = int(instruction[5:-1])
                qc2.z(qreg[old_qr]) #Applying phase flip error





In [ ]:
#Now let us introdcue the errors in the with the sifting and QBER in BB84 protocol

In [4]:
def print_outcomes_in_reserve(counts):
    for outcome in counts:
        reverse_outcome = ''
        for i in outcome:
            reverse_outcome = i + reverse_outcome

        return reverse_outcome

In [5]:
qreg = QuantumRegister(16) 
creg = ClassicalRegister(16)

send = []
asja_basis = []
balvis_basis=[]

#Asja 
asja = QuantumCircuit(qreg, creg, name = 'Asja')

for i in range(16):
    bit = randrange(2)
    send.append(bit)

for i,n in enumerate(send):
    if n==1:
        asja.x(qreg[i]) #Encoding the input according to the randomly generated message

#For randomly picking the basis
for i in range(16):
    r = randrange(2)
    if r == 0: #Encodes in Z basis
        asja_basis.append('Z')
    else: #Encodes in X basis
        asja.h(qreg[i])
        asja_basis.append('X')


balvis = QuantumCircuit(qreg, creg, name = 'Balvis') #Defining Balvis Circuit
NoisyChannel(asja, balvis, 'Asja') #Asja sends noisy states to Balvis


#Balvis
for i in range(16):
    r = randrange(2)
    if r == 0:
        balvis.measure(qreg[i],creg[i])
        balvis_basis.append('Z')

    else:
        balvis.h(qreg[i])
        balvis.measure(qreg[i],creg[i])
        balvis_basis.append('X')

job = execute(balvis, Aer.get_backend('qasm_simulator'), shots = 1)
counts = job.result().get_counts(balvis)
counts = print_outcomes_in_reserve(counts)
received = list(map(int,counts))



#Sifting
asja_key=[] #Asjas register for matching rounds
balvis_key=[] #Balvis register for matching rounds
for j in range(0,len(asja_basis)): #Going through list of bases 
    if asja_basis[j] == balvis_basis[j]: #Comparing
        asja_key.append(send[j])
        balvis_key.append(received[j]) #Keeping key bit if bases matched
    else:
        pass #Discard round if bases mismatched


#QBER
rounds = len(asja_key)//3
errors=0
for i in range(rounds):
    bit_index = randrange(len(asja_key)) 
    tested_bit = asja_key[bit_index]
    if asja_key[bit_index] != balvis_key[bit_index]: #comparing tested rounds
        errors=errors+1 #calculating errors
    del asja_key[bit_index] #removing tested bits from key strings
    del balvis_key[bit_index]
QBER=errors/rounds #calculating QBER
        
print("QBER value =", QBER)
print("Asja's secret key =", asja_key)
print("Balvis' secret key =", balvis_key)


QBER value = 0.0
Asja's secret key = [0, 0, 1, 1]
Balvis' secret key = [0, 0, 1, 1]


C:\Users\jrnat\AppData\Local\Temp\ipykernel_16088\1058832388.py:14: DeprecationWarning: The method ``qiskit.circuit.quantumcircuit.QuantumCircuit.qasm()`` is deprecated as of qiskit 0.46. It will be removed in the 1.0.0 release. Instead use the qiskit.qasm2.dump() or qiskit.qasm2.dumps() function
  qs = qc1.qasm().split(sep = ';')[4:-1]
C:\Users\jrnat\AppData\Local\Temp\ipykernel_16088\1575307452.py:45: DeprecationWarning: The function ``qiskit.execute_function.execute()`` is deprecated as of qiskit 0.46.0. It will be removed in the Qiskit 1.0 release. This function combines ``transpile`` and ``backend.run``, which is covered by ``Sampler`` :mod:`~qiskit.primitives`. Alternatively, you can also run :func:`.transpile` followed by ``backend.run()``.
  job = execute(balvis, Aer.get_backend('qasm_simulator'), shots = 1)
